In [42]:
import numpy as np
import os
import pickle as pickle
import json

from transformers import AutoTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [43]:
data_path = '../../data/'
context_name = "wikipedia_documents"
context_path = f"{context_name}.json"


In [44]:

model_name = "klue/bert-base"

tokenizer = AutoTokenizer.from_pretrained(
        model_name,
        use_fast=True,
    )

with open(os.path.join(data_path, context_path), "r", encoding="utf-8") as f:
    wiki = json.load(f)

contexts = list(
        dict.fromkeys([v["text"] for v in wiki.values()])
    )  # set 은 매번 순서가 바뀌므로
print(f"Lengths of unique contexts : {len(contexts)}")
ids = list(range(len(contexts)))

tfidfv = TfidfVectorizer(
        tokenizer=tokenizer.tokenize, ngram_range=(1, 2), max_features=50000,
    )
p_embedding = tfidfv.fit_transform(contexts)


/opt/conda/envs/boostcamp/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (1133 > 512). Running this sequence through the model will result in indexing errors


Lengths of unique contexts : 56737


In [45]:

train_dataset_path = f"{data_path}/train_dataset"
question_file = f"{data_path}/train_question.csv"

In [46]:
import json
def sep_indices(answer, indices):
    with open(f'../../data/{context_name}.json') as f:
        wiki = json.load(f)
    correct = []
    incorrect = []
    for cxt_idx in indices:
        if wiki[str(cxt_idx)]['document_id'] == cxt_idx:
            if answer in wiki[str(cxt_idx)]['text']:
                correct.append(cxt_idx)
            else:
                incorrect.append(cxt_idx)
        else:
            print('sorry')

    return correct, incorrect


In [47]:
## 질문ID / 질문 / 답 / 답이 포함되어 있는 지문 / 답이 포함되어있지 않은 지문
import pandas as pd

question_csv = pd.read_csv(question_file)

In [48]:

k = 20

In [49]:
doc_indices = []
doc_scores = []
tokenized_question = tfidfv.transform(question_csv['question'].values[0:10])
result = tokenized_question@p_embedding.T
if not isinstance(result, np.ndarray):
    result = result.toarray()
for i in range(result.shape[0]):
    sorted_result = np.argsort(result[i, :])[::-1]
    doc_scores.append(result[i, :][sorted_result].tolist()[:k])
    doc_indices.append(sorted_result.tolist()[:k])



In [50]:
from tqdm.auto import tqdm

result_df = pd.DataFrame([], columns = ["질문ID", "질문", "답", "답이 포함되어 있는 지문", "답이 포함되어있지 않은 지문"])

for q_data in tqdm(question_csv.values[0:10]):
    correct, incorrect = sep_indices(q_data[2], doc_indices[i])
    result_df.loc[len(result_df.index)] = [
        q_data[0], 
        q_data[1], 
        q_data[2], 
        correct, 
        incorrect
    ]

100%|██████████| 10/10 [00:15<00:00,  1.58s/it]


In [29]:
result_df.to_csv("resut.csv")


a = np.array([1.5, 0.2, 4.2, 2.5])
s = a.argsort()

print(s)
print(a[s[::-1]])
print(a[s][::-1])

In [51]:
import json

with open('../../data/wikipedia_documents.json') as f:
    wiki = json.load(f)

for i, q in enumerate(questions):
    print("+++++++++++++++++++++++question++++++++++++++++++++++++++++++")
    print(q)
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

    for cxt_idx in doc_indices[i]:
        if wiki[str(cxt_idx)]['document_id'] == cxt_idx:
            print(f"=========================={cxt_idx}===================================")
            print(wiki[str(cxt_idx)]['text'])
        else:
            print('sorry')

+++++++++++++++++++++++question++++++++++++++++++++++++++++++
대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
==========================52322===================================
레바논 베이루트에서 중동에 전문적이었던 미국인 학회원 맬컴 H. 커와 앤 즈위커 사이에 태어났다.  그는 3명의 형제, 자매를 두었다.  그의 조부 스탠리 커는 결국적으로 베이루트에 정착하기 전에 아르메니아인 집단학살 후 근동 구조와 함께 후원하고, 알레포와 마라시에서 여성과 고아들을 구출하였다.  커는 자신의 어린 시절을 레바논과 다른 중동의 나라들에서 보냈다.  그는 이집트에서 카이로 아메리칸 칼리지, 베이루트에서 아메리칸 커뮤니티 스쿨과 로스앤젤레스에 있는 팰리새디스 고등학교(현재의 팰리새디스 차터 고등학교)에서 수학하였다.

부친 맬컴은 베이루트에서 아메리칸 대학교에서 총장을 지내는 동안 1984년 1월 18일 52세의 나이로 레바논의 시아파 시민군 지하드에 의하여 살해되었다.  커는 당시 18세이며 칼리지의 신입생이었다.

1988년 커는 역사학, 사회학과 영어에 중요성과 함께 정통 학습 학사와 함께 애리조나 대학교를 졸업하였다.
==========================1738===================================
페아노 공리계(Peano公理系, Peano’s axioms)는 수리논리학에서 자연수 체계를 묘사하는 공리들이다. 수론의 일관성 및 완전성 연구에도 사용된다.

페아노의 공리들은 세 종류로 나눌 수 있다. 처음의 네 공리는 동일성에 대한 일반적인 명제로, 현대에는 보통 순수 논리의 공리로 취급된다. 다음의 네 공리는 따름수 연산의 근본적인 성질들을 자연수에 대한 1차 논리적 명제로 표현한 것이다. 마지막 9번째 공리는